In [42]:
import numpy as np
import pandas as pd
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from collections import Counter

In [43]:
df = pd.read_excel('Datasets_Ready.xlsx')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4510 non-null   int64  
 1   Text            4510 non-null   object 
 2   emoji           4510 non-null   object 
 3   Tokenized       4510 non-null   object 
 4   final_text      4494 non-null   object 
 5   text_emoji      4510 non-null   object 
 6   Pos_Word        4510 non-null   int64  
 7   Neg_Word        4510 non-null   int64  
 8   Total_Word      4510 non-null   int64  
 9   Pos_Ratio       4510 non-null   float64
 10  Neg_Ratio       4510 non-null   float64
 11  Sentimen_Text   4510 non-null   object 
 12  Tokenize_Emoji  4510 non-null   object 
 13  Pos_Emoji       4510 non-null   int64  
 14  Neg_Emoji       4510 non-null   int64  
 15  Sentimen_Emoji  4510 non-null   object 
 16  Sarcasm         4510 non-null   object 
dtypes: float64(2), int64(6), object(9

,Unnamed: 0,Text,emoji,Tokenized,final_text,text_emoji,Pos_Word,Neg_Word,Total_Word,Pos_Ratio,Neg_Ratio,Sentimen_Text,Tokenize_Emoji,Pos_Emoji,Neg_Emoji,Sentimen_Emoji,Sarcasm
0,0,kawan2 sunda bersatu jemput arteria dahlan,😁,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",kawan sunda bersatu jemput arteria dahlan,kawan sunda bersatu jemput arteria dahlan 😁,2,0,6,0.333333,0.000000,Positif,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",1,0,Positif,Negatif
1,2,arteria dahlan disidang adat sunda daerah menu...,😁,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",arteria dahlan sidang adat sunda daerah menunt...,arteria dahlan sidang adat sunda daerah menunt...,5,4,9,0.555556,0.444444,Positif,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",1,0,Positif,Negatif
2,4,ga ditafsirkan membanding bandingkan edy arter...,😁,"['ga', 'tafsir', 'membanding', 'membandingkan'...",ga tafsir membanding membandingkan edy arteria...,ga tafsir membanding membandingkan edy arteria...,3,2,9,0.333333,0.222222,Positif,"['ga', 'tafsir', 'membanding', 'membandingkan'...",1,0,Positif,Negatif
3,5,nantik ngak divhumas polri nya bilang palsu td...,😁,"['nanti', 'tidak', 'divisi humas', 'polri', '....",divisi humas polri bilang palsu mengeluarkan a...,divisi humas polri bilang palsu mengeluarkan a...,4,8,16,0.250000,0.500000,Negatif,"['divisi', 'humas', 'polri', 'bilang', 'palsu'...",1,0,Positif,Positif
4,6,urut dada dech kuasa hukum tdk mengerti hukum ...,😁,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'tid...",urut dada deh kuasa hukum mengerti hukum arter...,urut dada deh kuasa hukum mengerti hukum arter...,14,6,28,0.500000,0.214286,Positif,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'men...",1,0,Positif,Negatif


In [44]:
# Remove unused column/feature
df = df.drop(columns='Unnamed: 0')
df = df.drop(columns='Text')
df = df.drop(columns='emoji')
df = df.drop(columns='Tokenized')
df = df.drop(columns='final_text')
df = df.drop(columns='Pos_Word')
df = df.drop(columns='Neg_Word')
df = df.drop(columns='Total_Word')
df = df.drop(columns='Pos_Ratio')
df = df.drop(columns='Neg_Ratio')
df = df.drop(columns='Tokenize_Emoji')
df = df.drop(columns='Pos_Emoji')
df = df.drop(columns='Neg_Emoji')
df = df.drop(columns='Sentimen_Emoji')

In [45]:
df.head()

,text_emoji,Sentimen_Text,Sarcasm
0,kawan sunda bersatu jemput arteria dahlan 😁,Positif,Negatif
1,arteria dahlan sidang adat sunda daerah menunt...,Positif,Negatif
2,ga tafsir membanding membandingkan edy arteria...,Positif,Negatif
3,divisi humas polri bilang palsu mengeluarkan a...,Negatif,Positif
4,urut dada deh kuasa hukum mengerti hukum arter...,Positif,Negatif


In [46]:
df_ready = df[:20]

In [47]:
df_ready['Sarcasm'].value_counts()

Negatif    13
Positif     7
Name: Sarcasm, dtype: int64

In [48]:
df_ready['Sentimen_Text'] = df_ready['Sentimen_Text'].replace('Positif', 1)
df_ready['Sentimen_Text'] = df_ready['Sentimen_Text'].replace('Negatif', 0)

df_ready['Sarcasm'] = df_ready['Sarcasm'].replace('Positif', 1)
df_ready['Sarcasm'] = df_ready['Sarcasm'].replace('Negatif', 0)

C:\Users\DELL\AppData\Local\Temp\ipykernel_6756\3562434511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready['Sentimen_Text'] = df_ready['Sentimen_Text'].replace('Positif', 1)
C:\Users\DELL\AppData\Local\Temp\ipykernel_6756\3562434511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ready['Sentimen_Text'] = df_ready['Sentimen_Text'].replace('Negatif', 0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_6756\3562434511.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [49]:
df_ready['Sarcasm'].value_counts()

0    13
1     7
Name: Sarcasm, dtype: int64

In [50]:
print(Counter(df_ready['Sarcasm']))

Counter({0: 13, 1: 7})


In [51]:
max_len = 50
len_voc = 40000

In [52]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_ready, test_size=0.5)

In [53]:
# Tokenizing
def make_tokenizer(texts, len_voc):
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer(num_words=len_voc)
    t.fit_on_texts(texts)
    return t

In [54]:
tokenizer = make_tokenizer(df_ready['text_emoji'], len_voc)

In [55]:
tokenizer.word_index

{'arteria': 1,
 '😁': 2,
 'dahlan': 3,
 'sunda': 4,
 'hukum': 5,
 'goblok': 6,
 'maaf': 7,
 'masyarakat': 8,
 'jabar': 9,
 'indonesia': 10,
 'pejabat': 11,
 'bahasa': 12,
 'mak': 13,
 'polisi': 14,
 'adat': 15,
 'ga': 16,
 'polri': 17,
 'plat': 18,
 'tsb': 19,
 'memaafkan': 20,
 'partai': 21,
 'demokrasi': 22,
 'perjuangan': 23,
 'mendadak': 24,
 'lu': 25,
 'takut': 26,
 'doang': 27,
 'komentar': 28,
 'kawan': 29,
 'bersatu': 30,
 'jemput': 31,
 'sidang': 32,
 'daerah': 33,
 'menuntut': 34,
 'tafsir': 35,
 'membanding': 36,
 'membandingkan': 37,
 'edy': 38,
 'menegakkan': 39,
 'divisi': 40,
 'humas': 41,
 'bilang': 42,
 'palsu': 43,
 'mengeluarkan': 44,
 'membenarkan': 45,
 'no': 46,
 'asli': 47,
 'urut': 48,
 'dada': 49,
 'deh': 50,
 'kuasa': 51,
 'mengerti': 52,
 'dengar': 53,
 'pendapat': 54,
 'gedung': 55,
 'dpr': 56,
 'memiliki': 57,
 'kekebalan': 58,
 'atur': 59,
 'konstitusi': 60,
 'n': 61,
 'uu': 62,
 'kubu': 63,
 'saran': 64,
 'dorong': 65,
 'menangkap': 66,
 'semoga': 67,
 'be

In [56]:
X_train = tokenizer.texts_to_sequences(df_train['text_emoji'])
X_test = tokenizer.texts_to_sequences(df_test['text_emoji'])

In [57]:
from keras.utils import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

In [58]:
y_train = df_train['Sarcasm'].values
y_test = df_test['Sarcasm'].values

In [59]:
y_train

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1], dtype=int64)

In [60]:
y_test

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [61]:
stack_label = np.hstack((y_train, y_test))

In [62]:
stack_label.shape

(20,)

In [63]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index

In [64]:
def make_embedding_matrix(embedding, tokenizer, len_voc):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [65]:
glove = load_embedding('embeddings.txt')

In [66]:
embed_mat = make_embedding_matrix(glove, tokenizer, len_voc)

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [67]:
X_train_emb = embed_mat[X_train]
X_test_emb = embed_mat[X_test]

<p>SMOTE</p>

In [ ]:
from scipy.spatial.distance import cdist

In [ ]:
def calculate_distances(X_minority, X_majority):
    distances = cdist(X_minority, X_majority)
    return distances

In [ ]:
def get_neighbors(distances, k):
    neighbors = np.argsort(distances)[:, :k]
    return neighbors

In [ ]:
def generate_synthetic_samples(X_minority, neighbors):
    synthetic_samples = []
    for i in range(len(X_minority)):
        nn = np.random.choice(neighbors[i])
        alpha = np.random.uniform(0, 1)
        synthetic_sample = X_minority[i] + alpha * (X_minority[nn] - X_minority[i])
        synthetic_samples.append(synthetic_sample)
    synthetic_samples = np.array(synthetic_samples)
    return synthetic_samples

In [ ]:
def smote(X_minority, y_minority, k):
    minority_size = len(df_ready[df_ready['Sarcasm']==1])
    majority_size = len(df_ready[df_ready['Sarcasm']==0])
    
    # Menghitung jumlah tetangga terdekat berdasarkan perbandingan antara jumlah sampel minoritas dan mayoritas
    k_neighbors = int((majority_size / minority_size) * k)

    # Menghitung jarak antara sampel minoritas dan mayoritas
    distances = calculate_distances(X_minority, X_minority)
    
    # Mendapatkan tetangga terdekat untuk setiap sampel minoritas
    neighbors = get_neighbors(distances, k_neighbors)
    
    # Menghasilkan sampel sintetis
    synthetic_samples = generate_synthetic_samples(X_minority, neighbors)
    
    # Menggabungkan sampel minoritas asli dengan sampel sintetis
    X_oversampled = np.vstack((X_minority, synthetic_samples))
    y_oversampled = np.hstack((y_minority, np.ones(len(synthetic_samples))))
    
    return X_oversampled, y_oversampled

In [ ]:
X_minority = X_train[y_train == 1]

In [ ]:
X_oversampled, y_oversampled = smote(X_minority, X_sentiment_minority, y_minority)

In [ ]:
# train_size, max_len, embed_size = X_train_emb.shape
# X_train_emb_r = X_train_emb.reshape(train_size, max_len*embed_size)

In [ ]:
# test_size, max_len, embed_size = X_test_emb.shape
# X_test_emb_r = X_test_emb.reshape(test_size, max_len*embed_size)

In [ ]:
# stack = np.vstack((X_train_emb_r, X_test_emb_r))

In [ ]:
# stack.shape

In [ ]:
# from imblearn.over_sampling import SMOTE

In [ ]:
# smt = SMOTE()
# df_smote = smt.fit_resample(stack, stack_label)

In [ ]:
# df_smote

In [ ]:
# df_smote[0].shape

In [ ]:
# unique_values, value_counts = np.unique(df_smote[1], return_counts=True)

# print(unique_values)
# print(value_counts)